resize selenium window?

In [ ]:

var rows = 6;
var columns = 6;
var screen;

var getScreenSize = () => {
    return client
        .windowHandle()
        .then(r => client.windowHandleMaximize(r.value))
        .catch(e => {})
        .windowHandleSize()
        .then(r => (screen = r.value))
        .catch(e => console.log(e))
}

var resizeWindow = (i = 0) => {
    i = Math.abs(i + (typeof OFFSET !== 'undefined' ? OFFSET : 0));
    var hwnd;
    return client
        .windowHandle()
        .then(r => client.window((hwnd = r.value)))
        .then(() => getScreenSize())
        .then(screen => {
            const newPosition = {
                x: Math.floor(i % columns)
                    * Math.floor(screen.width / columns),
                y: Math.floor(i / columns % rows)
                    * Math.floor(screen.height / rows)
            };
            console.log(newPosition);
            return client.windowHandlePosition(hwnd, newPosition)
        })
        .then(() => client.windowHandleSize(hwnd, {
            width: Math.floor(screen.width / columns),
            height: Math.floor(screen.height / rows)
        }))
        .catch(e => console.log(e))
}

if(typeof client.resizeWindow === 'undefined') {
    client.addCommand('resizeWindow', resizeWindow);
}
module.exports = resizeWindow;


only one window?



In [ ]:
var importer = require('../Core');

var closeAllTabs = (keep) => {
    return client
        .getTabIds()
        .then(h => {
            var promises = [];
            for(var i = 0; i < h.length; i++) {
                if(h[i] === keep) {
                    continue;
                }
                promises.push((i => resolve => client
                              .switchTab(h[i])
                              .close()
                              .then(() => resolve())
                              .catch(e => resolve()))
                              .apply(this, [i]));
            }
            return importer.runAllPromises(promises);
        })
        .catch(e => console.log(e))
}

var closeAllWindows = (keep) => {
    return client
        .windowHandles()
        .then(h => {
            var promises = [];
            for(var i = 0; i < h.value.length; i++) {
                if(h.value[i] === keep) {
                    continue;
                }
                promises.push((i => resolve => client
                              .window(h.value[i])
                              .close()
                              .then(() => resolve())
                              .catch(e => resolve()))
                              .apply(this, [i]));
            }
            return importer.runAllPromises(promises);
        })
        .window(keep)
        .catch(e => console.log(e))
}

var onlyOneWindow = () => {
    return client
        .windowHandle()
        .then(r => client.window(r.value))
        .newWindow('https://google.com')
        .windowHandle()
        .then(r => closeAllWindows(r.value))
        .getCurrentTabId()
        .then(r => closeAllTabs(r))
        .resizeWindow(0)
        .catch(e => console.log(e))
    // TODO: close all tabs
}
module.exports = onlyOneWindow;



tile selenium chrome windows?



In [ ]:
var importer = require('../Core');

var openUrl = (url) => {
    var handles;
    return client
        .windowHandles()
        .then(h => handles = h.value)
        .execute((url, width, height) => {
            window.open(url, '', 'width=1350,height=1024');
        }, url)
        .windowHandles()
        .then(h => h.value.filter(hwnd => handles.indexOf(hwnd) === -1)[0])
        .catch(e => console.log(e))
    /*
    // TODO: use this openSPA()
    var id = Math.random().toString(36).substring(7);
    return client
        .execute((url, id, width, height) => {
            var a = document.createElement('a');
            a.setAttribute('onclick', 'window.open("' + url + '", "", "width=' + width + ',height=' + height + '"); return false;');
            a.setAttribute('id', id);
            a.href = url;
            a.style.position = 'absolute';
            a.style.zIndex = 4294967295;
            a.style.top = 0;
            a.style.left = 0;
            a.style.bottom = 0;
            a.style.right = 0;
            document.body.appendChild(a);
            setTimeout(() => a.remove(), 200);
        }, url, id, Math.floor(screen.width / columns), Math.floor(screen.height / rows))
        .click('a[id="' + id + '"]')
        */
}

var createNewWindows = (urls) => {
    var hwnd;
    return client
        .windowHandle()
        .then(r => hwnd = r.value)
        .windowHandles()
        .then(r => {
            // -1 first window is already used by sliced url
            const count = urls.length - (r.value.length - 1);
            const promises = [];
            for(var i = 0; i < count; i++) {
                promises.push((i => resolve => {
                    console.log(urls[i]);
                    return openUrl(urls[i])
                        .window(hwnd)
                        .then(h => resolve(h))
                        .catch(e => { console.log(e); resolve() })
                }).apply(this, [i]));
            }
            return importer.runAllPromises(promises);
        })
        .windowHandles()
        // position all windows after opening
        .then(r => {
            const promises = [];
            for(var i = 0; i < r.value.length; i++) {
                promises.push((i => resolve => {
                    return client
                        .window(r.value[i])
                        .resizeWindow(i)
                        .then(() => resolve())
                        .catch(e => {console.log(e); resolve()})
                }).apply(this, [i]));
            }
            return importer.runAllPromises(promises);
        })
        .catch(e => console.log(e))
}

var resizeWindow;
var tileWindows = (urls) => {
    return client
        .resizeWindow(0)
        .url(typeof urls === 'string' ? urls : urls[0])
        .then(h => typeof urls === 'string' ? [] : createNewWindows(urls.slice(1)))
        .windowHandles()
        .catch(e => console.log(e))
}
module.exports = tileWindows;



In [6]:
var importer = require('../Core');
var runSeleniumCell = importer.import('selenium cell');

var query = 'list of search engines';

$$.async();
//importer.import('child process')
    //.then(execCmd => Promise.all([
    //    execCmd('node -e "require(\'/Users/briancullinan/jupytangular2/Core\').import(\'run todays calendar events\').then(runner => runner()).then(e => process.exit(e)).catch(e=>{console.log(e); process.exit(e) });"'),
    //    execCmd('node -e "require(\'/Users/briancullinan/jupytangular2/Core\').import(\'run todays calendar events\').then(runner => runner()).then(e => process.exit(e)).catch(e=>{console.log(e); process.exit(e) });"')
    //]))
// get multiple sets of 9 working for single page scraping in parallel
runSeleniumCell('tile chrome windows')
    .then(tileWindows => tileWindows([
        'https://www.google.com/search?q=' + query,
        'https://www.bing.com/search?q=' + query,
        'https://search.yahoo.com/search?p=' + query,
        'https://www.ask.com/web?q=' + query,
        'https://search.aol.com/aol/search?q=' + query,
        'https://www.baidu.com/s?wd=' + query,
        'https://www.wolframalpha.com/input/?i=' + query,
        'https://duckduckgo.com/?q=' + query,
        'https://archive.org/search.php?query=' + query,
    ]))
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))



Initializing webdriver on localhost
[00:42:56]  COMMAND	GET 	 "/wd/hub/session/1c8ba9c91f146851229328caa33b824c"
[00:42:56]  COMMAND	GET 	 "/wd/hub/session/187aa56fb32a9a1891ffafbc84fcd3b9"
[00:42:56]  COMMAND	GET 	 "/wd/hub/session/65e30a37d1acb20f724dcc385caa946f"
[00:42:56]  COMMAND	GET 	 "/wd/hub/session/714a4207c3a5f6445fe605a96f4a8f34"
[00:42:56]  COMMAND	GET 	 "/wd/hub/session/a59fe47b233f672a6fc9a99ada843b55"
[00:42:56]  COMMAND	GET 	 "/wd/hub/session/71c998f11914595596204040e2b1e9a1"
[00:42:56]  COMMAND	GET 	 "/wd/hub/session/430ac2684023959913fd14651a51da81"
[00:42:56]  COMMAND	GET 	 "/wd/hub/session/88de7e501747214b58465b1f9c7c3709"
[00:42:56]  COMMAND	GET 	 "/wd/hub/session/ca32e065666fa9eaa09590f78ac5020d"
[00:42:56]  COMMAND	GET 	 "/wd/hub/session/e0af459268f289968f85289859505010"
[00:42:56]  COMMAND	GET 	 "/wd/hub/session/16a3d4fdae4a12d0dfb128185509f8f6"
[00:42:56]  COMMAND	GET 	 "/wd/hub/session/615e1cab2a14480e424fdb4928a47616"
[ 'e0af459268f289968f85289859505010',
  

{ sessionId: 'e0af459268f289968f85289859505010',
  status: 0,
  value: 
   [ 'CDwindow-be0e0feb-ed90-44f6-9c6f-ecbc7f9ccc6f',
     'CDwindow-7399c95d-c2ca-47e7-8ef6-860691c45b55',
     'CDwindow-7123e6b7-930d-40af-864a-3184b305bbcf',
     'CDwindow-8e86a763-5b4c-407a-ae59-635359a1c6ba',
     'CDwindow-dcfea344-7f8a-426b-b141-15cde37401e6',
     'CDwindow-81b2cd1d-ec04-48ed-80b1-fd2c17999920',
     'CDwindow-72509f30-6ac5-4f60-8cb8-acf1401e0b94',
     'CDwindow-d809b613-fa21-4d51-8d0a-b4627d29dba8',
     'CDwindow-985457c4-c87c-4310-9a4d-3766c1008e3a' ] }

get all session and window urls?



In [17]:
var getAllSessionUrls = () => {
    var windowCounter = -1;
    var session = client.requestHandler.sessionID;
    return client
        .getActiveSessions()
        .then(r => {
            return importer.runAllPromises(r.map(s => resolve => {
                client.requestHandler.sessionID = s;
                return client
                    .getUrl()
                    .windowHandles()
                    .then(h => importer.runAllPromises(h.value.map(hwnd => resolve => {
                        windowCounter++
                        return client
                            .window(hwnd)
                            .then(() => resizeWindow(windowCounter))
                            .getUrl()
                            .then(r => resolve(r))
                            .catch(e => console.log(e))
                    })))
                    .then(r => resolve(r))
                    .catch(e => console.log(e))
            }))
        })
        .catch(e => console.log(e))
}
module.exports = getAllSessionUrls;


[Function: getAllSessionUrls]

In [4]:
var importer = require('../Core');
var runSeleniumCell = importer.import('selenium cell');

$$.async();
runSeleniumCell(['resize selenium window', 'get all session urls'], false)
    .then(r => r.getAllSessionUrls())
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))


Initializing webdriver on localhost
[12:44:26]  COMMAND	GET 	 "/wd/hub/session/4dad63f68a0344d4ba0ade179584ceea"
[12:44:29]  COMMAND	GET 	 "/wd/hub/session/615e1cab2a14480e424fdb4928a47616"
[12:44:29]  COMMAND	GET 	 "/wd/hub/session/430ac2684023959913fd14651a51da81"
[12:44:30]  COMMAND	GET 	 "/wd/hub/session/16a3d4fdae4a12d0dfb128185509f8f6"
[12:44:30]  COMMAND	GET 	 "/wd/hub/session/e0af459268f289968f85289859505010"
[12:44:30]  COMMAND	GET 	 "/wd/hub/session/ca32e065666fa9eaa09590f78ac5020d"
[12:44:30]  COMMAND	GET 	 "/wd/hub/session/88de7e501747214b58465b1f9c7c3709"
[12:44:30]  COMMAND	GET 	 "/wd/hub/session/187aa56fb32a9a1891ffafbc84fcd3b9"
[12:44:30]  COMMAND	GET 	 "/wd/hub/session/71c998f11914595596204040e2b1e9a1"
[12:44:30]  COMMAND	GET 	 "/wd/hub/session/a59fe47b233f672a6fc9a99ada843b55"
[12:44:30]  COMMAND	GET 	 "/wd/hub/session/714a4207c3a5f6445fe605a96f4a8f34"
[12:44:30]  COMMAND	GET 	 "/wd/hub/session/65e30a37d1acb20f724dcc385caa946f"
[12:44:30]  COMMAND	GET 	 "/wd/hub/sessi

[ 'https://www.google.com/_/chrome/newtab?espv=2&ie=UTF-8',
  'https://www.facebook.com/emmanuchauvin/posts/1801321429884869',
  'https://www.facebook.com/',
  'https://www.linkedin.com/authwall?trk=ripf&trkInfo=AQEUcl90qSc5wwAAAV-LSkvYFmDos2BhI7bkc1wRbQPTdcDNhNUl7ym8PiuBfWvOwyl_KPGLCv3o2dcUY68MViFlUBf6J7TmRnlInJMjTgANZmjqkdf4KnJFT4SOYyRxzfwD0sc=&originalReferer=https://www.linkedin.com/&sessionRedirect=https%3A%2F%2Fwww.linkedin.com%2Fin%2Fkimberly-rizzo-63456971%2F',
  'https://www.linkedin.com/authwall?trk=ripf&trkInfo=AQFKoX-i8m0s-AAAAV-LUSXATOPHRWU7x7ldxNbR8YiM6jab6zAoQERHx74ciL7UB9fMizMzJqTSAk1I192_FAk8XAwGKa4EhJAUkat757BQo2z_bjIDuN0upTODfZ67QGnlMyc=&originalReferer=https://www.google.com/&sessionRedirect=https%3A%2F%2Fwww.linkedin.com%2Fin%2Fgail-barnes-pmp%25C2%25AE-csm-mba-2987a11%2F',
  'https://www.linkedin.com/authwall?trk=ripf&trkInfo=AQHnbaeW1-KbRQAAAV-LUghQ4IHb0zdmptIAHRxhsOkT_ejv0GCQ4n40D20H_HCnSa0mGm9Gt3hpEhwBcleLJsLRtOTHyMWnglSoUj_4PHdNgNoBm4R1gKbnTJiASq5nXNg1sa0=&ori